## Problem Definition

The classification of musical genres is important because it helps organize and navigate the vast and diverse world of music, which is crucial for both practical and academic purposes. Practically, it enhances user experiences in streaming platforms, enabling personalized recommendations and efficient content discovery. It also supports music organizations by automating categorization, improving accessibility, and driving music-related business models.

# Using Random Forests

A Random Forest algorithm is ideal for musical genre classification because it handles high-dimensional data well and captures complex, non-linear relationships between features like tempo and melody. It is robust against overfitting by combining multiple decision trees, improving generalization to new songs. Additionally, it can handle noisy or missing data and provides insights into feature importance, making it a reliable and efficient choice for accurate genre classification.

In [8]:
pip install imbalanced-learn

In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

## Musical Genre Sorter

In [10]:
np.random.seed(42)
n_samples = 20000  # Number of samples

In [11]:
# Features: tempo, duration, energy, key, danceability, acousticness, valence
tempo = np.random.randint(80, 180, n_samples)  # Random tempo between 80 and 180 BPM
duration = np.random.randint(120, 300, n_samples)  # Random duration between 120s and 300s
energy = np.random.uniform(0.4, 1.0, n_samples)  # Random energy between 0.4 and 1.0
key = np.random.randint(0, 4, n_samples)  # Random key (0=Major, 1=Minor, 2=Blues, 3=Jazz)
danceability = np.random.uniform(0.4, 1.0, n_samples)  # Danceability (0-1 scale)
acousticness = np.random.uniform(0.0, 1.0, n_samples)  # Acousticness (0-1 scale)
valence = np.random.uniform(0.0, 1.0, n_samples)  # Valence (positive/negative emotions scale)

In [12]:
# Genre rules: Define rules based on genre features
genre_rules = {
    0: {  # Pop
        'name': 'pop',
        'conditions': [
            ('energy', 0.5, 'tempo', 120, 'key', 0),
            ('energy', 0.6, 'tempo', 130, 'key', 0),
            ('danceability', 0.7, 'valence', 0.8, 'key', 0)
        ]
    },
    1: {  # Rock
        'name': 'rock',
        'conditions': [
            ('energy', 0.6, 'tempo', 120, 'key', 1),
            ('energy', 0.7, 'tempo', 110, 'key', 2),
            ('energy', 0.8, 'tempo', 130, 'key', 1)
        ]
    },
    2: {  # Jazz
        'name': 'jazz',
        'conditions': [
            ('energy', 0.4, 'tempo', 100, 'key', 1),
            ('energy', 0.5, 'tempo', 110, 'key', 3),
            ('acousticness', 0.7, 'danceability', 0.5, 'key', 3)
        ]
    },
    3: {  # Hip Hop
        'name': 'hip-hop',
        'conditions': [
            ('energy', 0.7, 'tempo', 90, 'key', 2),
            ('danceability', 0.8, 'acousticness', 0.3, 'key', 2)
        ]
    },
    4: {  # Salsa
        'name': 'salsa',
        'conditions': [
            ('energy', 0.8, 'tempo', 110, 'key', 1),
            ('danceability', 0.9, 'acousticness', 0.5, 'key', 0)
        ]
    },
    5: {  # Blues
        'name': 'blues',
        'conditions': [
            ('energy', 0.5, 'tempo', 110, 'key', 2),
            ('danceability', 0.6, 'acousticness', 0.8, 'key', 2)
        ]
    },
    6: {  # Metal
        'name': 'metal',
        'conditions': [
            ('energy', 0.9, 'tempo', 140, 'key', 1),
            ('acousticness', 0.2, 'valence', 0.3, 'key', 1)
        ]
    },
    7: {  # Disco
        'name': 'disco',
        'conditions': [
            ('energy', 0.7, 'tempo', 120, 'key', 0),
            ('danceability', 0.9, 'valence', 0.9, 'key', 0)
        ]
    },
    8: {  # Folk
        'name': 'folk',
        'conditions': [
            ('energy', 0.5, 'tempo', 100, 'key', 0),
            ('acousticness', 0.8, 'valence', 0.7, 'key', 0)
        ]
    },
    9: {  # Funk
        'name': 'funk',
        'conditions': [
            ('energy', 0.7, 'tempo', 110, 'key', 0),
            ('danceability', 0.8, 'acousticness', 0.4, 'key', 0)
        ]
    },
    10: {  # Reggae
        'name': 'reggae',
        'conditions': [
            ('energy', 0.6, 'tempo', 80, 'key', 1),
            ('danceability', 0.8, 'acousticness', 0.7, 'key', 1)
        ]
    },
    11: {  # Country
        'name': 'country',
        'conditions': [
            ('energy', 0.6, 'tempo', 110, 'key', 0),
            ('acousticness', 0.7, 'valence', 0.6, 'key', 0)
        ]
    },
    12: {  # Experimental
        'name': 'experimental',
        'conditions': [
            ('energy', 0.4, 'tempo', 100, 'key', 3),
            ('acousticness', 0.9, 'valence', 0.5, 'key', 3)
        ]
    },
    13: {  # Punk
        'name': 'punk',
        'conditions': [
            ('energy', 0.9, 'tempo', 160, 'key', 1),
            ('acousticness', 0.3, 'valence', 0.4, 'key', 1)
        ]
    },
    14: {  # Latin
        'name': 'latin',
        'conditions': [
            ('energy', 0.8, 'tempo', 130, 'key', 0),
            ('danceability', 0.9, 'acousticness', 0.6, 'key', 0)
        ]
    },
    15: {  # R&B
        'name': 'r&b',
        'conditions': [
            ('energy', 0.5, 'tempo', 80, 'key', 0),
            ('danceability', 0.7, 'acousticness', 0.4, 'key', 0)
        ]
    },
    16: {  # Soul
        'name': 'soul',
        'conditions': [
            ('energy', 0.6, 'tempo', 90, 'key', 0),
            ('danceability', 0.7, 'valence', 0.8, 'key', 0)
        ]
    },
    17: {  # Indie
        'name': 'indie',
        'conditions': [
            ('energy', 0.6, 'tempo', 110, 'key', 1),
            ('acousticness', 0.8, 'valence', 0.7, 'key', 1)
        ]
    },
    18: {  # House
        'name': 'house',
        'conditions': [
            ('energy', 0.9, 'tempo', 120, 'key', 1),
            ('danceability', 0.9, 'acousticness', 0.5, 'key', 1)
        ]
    },
    19: {  # Trance
        'name': 'trance',
        'conditions': [
            ('energy', 0.9, 'tempo', 130, 'key', 1),
            ('danceability', 0.9, 'valence', 0.7, 'key', 1)
        ]
    },
    20: {  # EDM
        'name': 'edm',
        'conditions': [
            ('energy', 0.9, 'tempo', 140, 'key', 1),
            ('danceability', 0.9, 'valence', 0.8, 'key', 1)
        ]
    }
}

In [13]:
# function to apply genre rules for the model
def apply_genre_rules(row, genre_rules):
    for genre, rule in genre_rules.items():
        for condition in rule['conditions']:
            feature1, value1, feature2, value2, feature3, value3 = condition
            # Check if the row meets the conditions for the genre
            if (row[feature1] >= value1) and (row[feature2] >= value2) and (row[feature3] == value3):
                return genre
    return np.random.choice([i for i in range(len(genre_rules))])  # If no rule matches, return a random genre

# Apply genre rules to the data
genre_labels = np.array([apply_genre_rules(row, genre_rules) for index, row in pd.DataFrame({
    'tempo': tempo,
    'duration': duration,
    'energy': energy,
    'key': key,
    'danceability': danceability,
    'acousticness': acousticness,
    'valence': valence
}).iterrows()])

# Create DataFrame
data = pd.DataFrame({
    'tempo': tempo,
    'duration': duration,
    'energy': energy,
    'key': key,
    'danceability': danceability,
    'acousticness': acousticness,
    'valence': valence,
    'genre': genre_labels
})

In [ ]:
# Define features (X)
X = data[['tempo', 'duration', 'energy', 'key', 'danceability', 'acousticness', 'valence']]

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=5)
data['cluster'] = dbscan.fit_predict(X_scaled)
X_scaled_with_clusters = np.column_stack((X_scaled, data['cluster']))

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled_with_clusters, genre_labels, test_size=0.2, random_state=42)

# Handle class imbalance using class weights for the random forest
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

# SMOTE for balancing classes
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Define Random Forest
rf_model = RandomForestClassifier(
    n_estimators=200,          # Number of trees in the forest
    max_depth=15,              # Maximum depth of trees
    min_samples_split=5,       # Minimum samples to split a node
    min_samples_leaf=1,        # Minimum samples at a leaf node
    class_weight=class_weight_dict,  # Handling imbalance with class weights
    random_state=42
)

# Train the Random Forest model and evaluate on the test set
rf_model.fit(X_train_res, y_train_res)
y_pred = rf_model.predict(X_test)

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")


Accuracy: 0.85025
Confusion Matrix:
[[ 553    0    0    4    1    1    0    2    5    3    0    3    1    6
     1    5    6    1    0    4    2]
 [   0  756    0    2    0    1    0    4    0    0    2    1    0    1
     3    2    0    5    1    1    2]
 [   0    3 1007    1    1    0    1    5    0    1    0    3   16    4
     9    1    3   10   11    6   13]
 [   0    0    0  204    0    1    1    2    0    0    1    1    1    6
     2    1    3   12    6    7    9]
 [   0    0    0    0   41    0    0    1    0    4    1    0    0    0
     0    0    1    3    3    1    3]
 [   0    3    0    4    0  195    0    4    0    0    1    1    0    3
     0    0    5    5    3    5    9]
 [   0    0    0    1    0    0  101    0    0    0    2    0    0    3
     2    0    2    1    1    0    3]
 [   0    0    0    0    0    1    0    3    0    0    0    1    0    4
     1    0    1    4    1    2    2]
 [   1    0    0    0    0    2    0    2  135    3    0    3    0    1
     4    0 